In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold, cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_classif
from sklearn.pipeline import make_pipeline

import tensorflow as tf
import tensorflow.keras as keras
from keras import Model
from keras import Sequential
from keras import regularizers
from keras.layers import Dense, Concatenate, Input, BatchNormalization, Activation
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.optimizers import Adagrad, Adam
from keras.callbacks import EarlyStopping

import numpy as np

import itertools as it
from statistics import mean

Using TensorFlow backend.


# Data Processing #

In [2]:
# 1. Import data
train_data = pd.read_csv('imputed_training_data.csv')
df = pd.DataFrame(train_data)
df = df.ix[:,1:] # Remove index column

train_labels = pd.read_csv('train_labels.csv') # Last 4 columns are real-valued, everything else is boolean
df2 = pd.DataFrame(train_labels)

/home/carter/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
/home/carter/.local/lib/python3.6/site-packages/pandas/core/indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)


In [3]:
# 2. Analyze the data
#print(df.info())
print(df.describe())

# 2a. Handle missing data
def custom_impute(df):
    df_copy = df.copy()
    for pid in range(int(df.shape[0]/12)):
        row_start = pid*12

        for col_idx in range(34):
            col_idx += 3

            series = df.ix[row_start:row_start+12, col_idx]
            series = series.interpolate(method='pad')

            df_copy.ix[row_start:row_start+12, col_idx] = series 
        
    df_copy = df.fillna(df.median())
    df_copy.to_csv('imputed_stuff.csv')
    
    return df_copy

                 pid           Time            Age          EtCO2  \
count  227940.000000  227940.000000  227940.000000  227940.000000   
mean    15788.831219       7.014399      62.073809      32.389339   
std      9151.896286       4.716103      16.451854       9.245298   
min         1.000000       1.000000      15.000000      10.000000   
25%      7879.000000       4.000000      52.000000      27.000000   
50%     15726.000000       7.000000      64.000000      33.000000   
75%     23725.000000      10.000000      74.000000      38.000000   
max     31658.000000     315.000000     100.000000     100.000000   

                 PTT            BUN        Lactate           Temp  \
count  227940.000000  227940.000000  227940.000000  227940.000000   
mean       38.282748      23.025643       2.292870      36.807476   
std        24.110589      19.837206       1.882794       0.819596   
min        12.500000       1.000000       0.200000      21.000000   
25%        27.500000      12.0000

In [4]:
print(len(np.unique(df["pid"])))

18995


In [5]:
# 2.  Pre-processing
def feature_transform(df):
    
    pid_series = pd.unique(df["pid"])
    if "Time" in df.columns:
        df = df.drop("Time", axis=1)

    # Combine rows from a single patient, do not duplicate age
    num_patients = int(df.shape[0]/12)
    num_feats = int(df.shape[1]*2 - 2)

    df_np = df.to_numpy()
    df_np_new = np.zeros((num_patients, num_feats))

    for patient_idx in range(num_patients):
    ## Stack time-series information as feature
    #     df_np_new[patient_idx,:] = np.hstack((df_np[patient_idx,1:], df_np[patient_idx+1,2:], df_np[patient_idx+2,2:], 
    #                                 df_np[patient_idx+3,2::], df_np[patient_idx+4,2:], df_np[patient_idx+5,2:], 
    #                                 df_np[patient_idx+6,2::], df_np[patient_idx+7,2:], df_np[patient_idx+8,2:], 
    #                                 df_np[patient_idx+9,2::], df_np[patient_idx+10,2:], df_np[patient_idx+11,2:]))

        # Take mean and std as features
        curr_patient_data = df_np[patient_idx:patient_idx+12, np.arange(df.shape[1]-1)+1]
        patient_mean = np.mean(curr_patient_data, axis=0)
        patient_std = np.std(curr_patient_data, axis=0)
        df_np_new[patient_idx,:] = np.hstack((patient_mean, patient_std))

    return pid_series, df_np_new

pids_train, x_train = feature_transform(df)

# 2c. Separate training labels
if "pid" in df2.columns:
    df2 = df2.drop("pid", axis=1)
df2_np = df2.to_numpy()

# Separate labels into the sub-tasks
subtask1_labels = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 
                   'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
                   'LABEL_Bilirubin_direct', 'LABEL_EtCO2']

subtask2_labels = ['LABEL_Sepsis']

subtask3_labels = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

y_subtask1 = df2[subtask1_labels].to_numpy()
y_subtask2 = df2[subtask2_labels].to_numpy()
y_subtask3 = df2[subtask3_labels].to_numpy()

In [6]:
# 3. Remove features with low correlation
# x_train_new = SelectKBest(mutual_info_classif, k=5).fit_transform(x_train, y_train_cls[:,0])
# x_train = x_train_new

In [7]:
# 2d. Normalize
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)

from keras.callbacks import Callback

class RocCallback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred_train = self.model.predict(self.x)
        roc_train = roc_auc_score(self.y, y_pred_train)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc_train: %s - roc-auc_val: %s' % (str(round(roc_train,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [8]:
# Balancing using class-weights for loss
subtask1_class_weights = [compute_class_weight('balanced', np.unique(y_subtask1[:,i]), y_subtask1[:,i]) for i in range(y_subtask1.shape[1])]

for i in range(y_subtask1.shape[1]):
   max_ind = subtask1_class_weights[i].argmax()
   subtask1_class_weights[i][max_ind] *= 1

subtask1_class_weights = [dict(enumerate(weights)) for weights in subtask1_class_weights]

subtask2_class_weights = compute_class_weight('balanced', np.unique(y_subtask2.squeeze()), y_subtask2.squeeze())

In [ ]:
# Set Batch-size, Epochs (same across all 3)
batch_size = 2500
epochs = 100
folds = 5

# Subtask 1 #

## Architecture ##

In [ ]:
def subtask1_clf(lr, decay, alpha, num_layers, num_neurons):

    x = Input(shape=(x_train.shape[1],))
    layers = []
    last_layer = x
    for layer_num in range(1,num_layers+1):
        new_layer = Dense(num_neurons, 
                          activation='linear', 
                          kernel_regularizer=regularizers.l2(alpha),
                            kernel_initializer='he_normal')(last_layer)
        layers.append(new_layer)
        
        new_layer_activate = Activation('relu')(new_layer)
        layers.append(new_layer_activate)
        
        new_layer_norm = BatchNormalization()(new_layer_activate)
        layers.append(new_layer_norm)
        
        last_layer = new_layer_norm

    y = Dense(10, activation='sigmoid')(last_layer)
    
    model = Model(inputs=x, outputs=y)
    ada_grad = Adagrad(lr=lr, epsilon=1e-08, decay=decay)
    #adam_grad = Adam(lr=0.1, beta_1=0.9, beta_2=0.9)

    model.compile(optimizer=ada_grad, loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

subtask1_buildfn = KerasClassifier(subtask1_clf)

## Cross-validation ##

In [ ]:
learning_rates = np.logspace(-4,-1,5)
decay = np.logspace(-2,0,5)
alpha = np.logspace(-1,3,5)
num_layers = [2, 3, 4, 5]
num_neurons = [32, 64, 128]


param_grid = dict(lr=learning_rates, decay=decay,
                 alpha=alpha, num_layers = num_layers,
                 num_neurons=num_neurons)
    
grid = GridSearchCV(subtask1_buildfn, param_grid=param_grid, cv=folds, refit=False)
grid_result = grid.fit(x_train, y_subtask1, 
                       batch_size=batch_size, epochs=epochs,
                       class_weight=subtask1_class_weights)

# Unpack best params, retrain model
lr, decay, alpha, num_layers, num_neurons = [int(x) for x in grid_result.best_params_.values()]
subtask1_best_model = subtask1_clf(lr, decay, alpha, num_layers, num_neurons)

subtask1_history = subtask1_best_model.fit(x_train, y_subtask1,
                                            batch_size=batch_size, epochs=10*epochs,
                                            class_weight=subtask1_class_weights)

Epoch 1/100
15196/15196 [==============================] - 0s 28us/step - loss: 13.1793 - acc: 0.5002
Epoch 2/100
15196/15196 [==============================] - 0s 5us/step - loss: 13.1492 - acc: 0.5000
Epoch 3/100
15196/15196 [==============================] - 0s 5us/step - loss: 13.1314 - acc: 0.5000
Epoch 4/100
15196/15196 [==============================] - 0s 6us/step - loss: 13.1178 - acc: 0.5001
Epoch 5/100
15196/15196 [==============================] - 0s 5us/step - loss: 13.1067 - acc: 0.5003
Epoch 6/100
15196/15196 [==============================] - 0s 5us/step - loss: 13.0973 - acc: 0.5003
Epoch 7/100
15196/15196 [==============================] - 0s 5us/step - loss: 13.0893 - acc: 0.5004
Epoch 8/100
15196/15196 [==============================] - 0s 5us/step - loss: 13.0822 - acc: 0.5005
Epoch 9/100
15196/15196 [==============================] - 0s 5us/step - loss: 13.0759 - acc: 0.5006
Epoch 10/100
15196/15196 [==============================] - 0s 5us/step - loss: 13.0702 - 

15196/15196 [==============================] - 0s 4us/step - loss: 12.9686 - acc: 0.5010
Epoch 82/100
15196/15196 [==============================] - 0s 3us/step - loss: 12.9680 - acc: 0.5009
Epoch 83/100
15196/15196 [==============================] - 0s 3us/step - loss: 12.9677 - acc: 0.5006
Epoch 84/100
15196/15196 [==============================] - 0s 3us/step - loss: 12.9672 - acc: 0.5010
Epoch 85/100
15196/15196 [==============================] - 0s 3us/step - loss: 12.9667 - acc: 0.5010
Epoch 86/100
15196/15196 [==============================] - 0s 3us/step - loss: 12.9664 - acc: 0.5011
Epoch 87/100
15196/15196 [==============================] - 0s 3us/step - loss: 12.9659 - acc: 0.5008
Epoch 88/100
15196/15196 [==============================] - 0s 3us/step - loss: 12.9655 - acc: 0.5008
Epoch 89/100
15196/15196 [==============================] - 0s 3us/step - loss: 12.9650 - acc: 0.5006
Epoch 90/100
15196/15196 [==============================] - 0s 3us/step - loss: 12.9647 - acc: 

15196/15196 [==============================] - 0s 4us/step - loss: 13.6420 - acc: 0.4972
Epoch 61/100
15196/15196 [==============================] - 0s 3us/step - loss: 13.6412 - acc: 0.4966
Epoch 62/100
15196/15196 [==============================] - 0s 3us/step - loss: 13.6406 - acc: 0.4974
Epoch 63/100
15196/15196 [==============================] - 0s 3us/step - loss: 13.6398 - acc: 0.4972
Epoch 64/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.6392 - acc: 0.4971
Epoch 65/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.6387 - acc: 0.4970
Epoch 66/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.6379 - acc: 0.4976
Epoch 67/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.6374 - acc: 0.4968
Epoch 68/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.6368 - acc: 0.4972
Epoch 69/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.6361 - acc: 

15196/15196 [==============================] - 0s 3us/step - loss: 13.9393 - acc: 0.5008
Epoch 40/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.9381 - acc: 0.5010
Epoch 41/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.9367 - acc: 0.5012
Epoch 42/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.9356 - acc: 0.5010
Epoch 43/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.9347 - acc: 0.5010
Epoch 44/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.9336 - acc: 0.5012
Epoch 45/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.9324 - acc: 0.5019
Epoch 46/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.9314 - acc: 0.5010
Epoch 47/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.9304 - acc: 0.5010
Epoch 48/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.9294 - acc: 

15196/15196 [==============================] - 0s 4us/step - loss: 13.2003 - acc: 0.5145
Epoch 19/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.1974 - acc: 0.5146
Epoch 20/100
15196/15196 [==============================] - 0s 5us/step - loss: 13.1947 - acc: 0.5151
Epoch 21/100
15196/15196 [==============================] - 0s 5us/step - loss: 13.1921 - acc: 0.5147
Epoch 22/100
15196/15196 [==============================] - 0s 5us/step - loss: 13.1896 - acc: 0.5147
Epoch 23/100
15196/15196 [==============================] - 0s 5us/step - loss: 13.1872 - acc: 0.5145
Epoch 24/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.1851 - acc: 0.5148
Epoch 25/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.1829 - acc: 0.5143
Epoch 26/100
15196/15196 [==============================] - 0s 4us/step - loss: 13.1810 - acc: 0.5145
Epoch 27/100
15196/15196 [==============================] - 0s 5us/step - loss: 13.1789 - acc: 

...

15196/15196 [==============================] - 0s 12us/step - loss: 101.1863 - acc: 0.5008
Epoch 41/100
15196/15196 [==============================] - 0s 12us/step - loss: 101.1722 - acc: 0.5008
Epoch 42/100
15196/15196 [==============================] - 0s 12us/step - loss: 101.1589 - acc: 0.5009
Epoch 43/100
15196/15196 [==============================] - 0s 12us/step - loss: 101.1461 - acc: 0.5008
Epoch 44/100
15196/15196 [==============================] - 0s 12us/step - loss: 101.1335 - acc: 0.5002
Epoch 45/100
15196/15196 [==============================] - 0s 12us/step - loss: 101.1211 - acc: 0.5012
Epoch 46/100
15196/15196 [==============================] - 0s 12us/step - loss: 101.1096 - acc: 0.5014
Epoch 47/100
15196/15196 [==============================] - 0s 12us/step - loss: 101.0979 - acc: 0.5013
Epoch 48/100
15196/15196 [==============================] - 0s 12us/step - loss: 101.0866 - acc: 0.5003
Epoch 49/100
15196/15196 [==============================] - 0s 12us/step - lo

15196/15196 [==============================] - 0s 12us/step - loss: 101.4447 - acc: 0.5047
Epoch 19/100
15196/15196 [==============================] - 0s 12us/step - loss: 101.4104 - acc: 0.5047
Epoch 20/100
15196/15196 [==============================] - 0s 12us/step - loss: 101.3779 - acc: 0.5044
Epoch 21/100
15196/15196 [==============================] - 0s 12us/step - loss: 101.3467 - acc: 0.5045
Epoch 22/100
15196/15196 [==============================] - 0s 12us/step - loss: 101.3175 - acc: 0.5045
Epoch 23/100
15196/15196 [==============================] - 0s 12us/step - loss: 101.2899 - acc: 0.5047
Epoch 24/100
15196/15196 [==============================] - 0s 12us/step - loss: 101.2635 - acc: 0.5045
Epoch 25/100
15196/15196 [==============================] - 0s 12us/step - loss: 101.2384 - acc: 0.5044
Epoch 26/100
15196/15196 [==============================] - 0s 12us/step - loss: 101.2146 - acc: 0.5052
Epoch 27/100
15196/15196 [==============================] - 0s 12us/step - lo

15196/15196 [==============================] - 0s 12us/step - loss: 100.5355 - acc: 0.5055
Epoch 97/100
15196/15196 [==============================] - 0s 12us/step - loss: 100.5309 - acc: 0.5051
Epoch 98/100
15196/15196 [==============================] - 0s 12us/step - loss: 100.5264 - acc: 0.5054
Epoch 99/100
15196/15196 [==============================] - 0s 12us/step - loss: 100.5226 - acc: 0.5049
Epoch 100/100
3799/3799 [==============================] - 3s 808us/step
Epoch 1/100
15196/15196 [==============================] - 8s 501us/step - loss: 103.0452 - acc: 0.5022
Epoch 2/100
15196/15196 [==============================] - 0s 12us/step - loss: 102.6848 - acc: 0.5030
Epoch 3/100
15196/15196 [==============================] - 0s 12us/step - loss: 102.4695 - acc: 0.5028
Epoch 4/100
15196/15196 [==============================] - 0s 12us/step - loss: 102.3064 - acc: 0.5030
Epoch 5/100
15196/15196 [==============================] - 0s 12us/step - loss: 102.1740 - acc: 0.5028
Epoch 6/

15196/15196 [==============================] - 0s 12us/step - loss: 100.5600 - acc: 0.5050
Epoch 75/100
15196/15196 [==============================] - 0s 12us/step - loss: 100.5538 - acc: 0.5041
Epoch 76/100
15196/15196 [==============================] - 0s 12us/step - loss: 100.5477 - acc: 0.5048
Epoch 77/100
15196/15196 [==============================] - 0s 12us/step - loss: 100.5417 - acc: 0.5050
Epoch 78/100
15196/15196 [==============================] - 0s 12us/step - loss: 100.5358 - acc: 0.5051
Epoch 79/100
15196/15196 [==============================] - 0s 12us/step - loss: 100.5302 - acc: 0.5046
Epoch 80/100
15196/15196 [==============================] - 0s 12us/step - loss: 100.5245 - acc: 0.5045
Epoch 81/100
15196/15196 [==============================] - 0s 12us/step - loss: 100.5192 - acc: 0.5047
Epoch 82/100
15196/15196 [==============================] - 0s 12us/step - loss: 100.5136 - acc: 0.5053
Epoch 83/100
15196/15196 [==============================] - 0s 12us/step - lo

15196/15196 [==============================] - 0s 18us/step - loss: 100.3787 - acc: 0.5078
Epoch 53/100
15196/15196 [==============================] - 0s 14us/step - loss: 100.3690 - acc: 0.5074
Epoch 54/100
15196/15196 [==============================] - 0s 12us/step - loss: 100.3591 - acc: 0.5072
Epoch 55/100
15196/15196 [==============================] - 0s 13us/step - loss: 100.3499 - acc: 0.5084
Epoch 56/100
15196/15196 [==============================] - 0s 12us/step - loss: 100.3409 - acc: 0.5084
Epoch 57/100
15196/15196 [==============================] - 0s 12us/step - loss: 100.3317 - acc: 0.5076
Epoch 58/100
15196/15196 [==============================] - 0s 12us/step - loss: 100.3234 - acc: 0.5088
Epoch 59/100
15196/15196 [==============================] - 0s 12us/step - loss: 100.3149 - acc: 0.5079
Epoch 60/100
15196/15196 [==============================] - 0s 13us/step - loss: 100.3065 - acc: 0.5081
Epoch 61/100
15196/15196 [==============================] - 0s 12us/step - lo

# Subtask 2 #

## Architecture ##

In [ ]:
def subtask2_clf(lr, decay, alpha, num_layers, num_neurons):

    x = Input(shape=(x_train.shape[1],))
    layers = []
    last_layer = x
    for layer_num in range(1,num_layers+1):
        new_layer = Dense(num_neurons, 
                          activation='linear', 
                          kernel_regularizer=regularizers.l2(alpha),
                            kernel_initializer='he_normal')(last_layer)
        layers.append(new_layer)
        
        new_layer_activate = Activation('relu')(new_layer)
        layers.append(new_layer_activate)
        
        new_layer_norm = BatchNormalization()(new_layer_activate)
        layers.append(new_layer_norm)
        
        last_layer = new_layer_norm

    y = Dense(1, activation='sigmoid')(last_layer)
    
    model = Model(inputs=x, outputs=y)
    ada_grad = Adagrad(lr=lr, epsilon=1e-08, decay=decay)
    #adam_grad = Adam(lr=0.1, beta_1=0.9, beta_2=0.9)

    model.compile(optimizer=ada_grad, loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

subtask2_buildfn = KerasClassifier(subtask2_clf)

## Cross-Validation ##

In [ ]:
# Subtask 2 
learning_rates = np.logspace(-4,-1,5)
decay = np.logspace(-2,0,5)
alpha = np.logspace(-1,3,5)
num_layers = [2, 3, 4, 5]
num_neurons = [32, 64, 128]

param_grid2 = dict(lr=learning_rates, decay=decay,
                 alpha=alpha, num_layers = num_layers,
                 num_neurons=num_neurons)
    
grid2 = GridSearchCV(subtask2_buildfn, param_grid=param_grid2, cv=folds, refit=False)
grid2_result = grid2.fit(x_train, y_subtask2, 
                         batch_size=batch_size, epochs=epochs,
                         class_weight=subtask2_class_weights)

# Unpack best params, retrain model
lr, decay, alpha, num_layers, num_neurons = [int(x) for x in grid_result2.best_params_.values()]
subtask2_best_model = subtask2_clf(lr, decay, alpha, num_layers, num_neurons)

subtask2_history = subtask2_best_model.fit(x_train, y_subtask2,
                                            batch_size=batch_size, epochs=10*epochs,
                                            class_weight=subtask2_class_weights)

# Subtask 3 #

## Architecture ##

In [ ]:
def subtask3_reg(lr, decay, alpha, num_layers, num_neurons):
    x = Input(shape=(x_train.shape[1],))
    layers = []
    last_layer = x
    for layer_num in range(1,num_layers+1):
        new_layer = Dense(num_neurons, 
                          activation='linear', 
                          kernel_regularizer=regularizers.l2(alpha),
                            kernel_initializer='he_normal')(last_layer)
        layers.append(new_layer)
        
        new_layer_activate = Activation('relu')(new_layer)
        layers.append(new_layer_activate)
        
        new_layer_norm = BatchNormalization()(new_layer_activate)
        layers.append(new_layer_norm)
        
        last_layer = new_layer_norm

    y = Dense(4, activation='linear')(last_layer)

    # To-do: Insert outputs for regression outputs
    model = Model(inputs=x, outputs=y)
    ada_grad = Adagrad(lr=lr, epsilon=1e-08, decay=decay)
    #adam_grad = Adam(lr=0.1, beta_1=0.9, beta_2=0.9)

    model.compile(optimizer=ada_grad, loss='mean_squared_error',
                  metrics=['mse'])

    return model

subtask3_buildfn = KerasRegressor(subtask3_reg)

## Cross-Validation ##

In [ ]:
# Subtask 3 
learning_rates = np.logspace(-4,-1,5)
decay = np.logspace(-2,0,5)
alpha = np.logspace(-1,3,5)
num_layers = [2, 3, 4, 5]
num_neurons = [32, 64, 128]

param_grid3 = dict(lr=learning_rates, decay=decay,
                 alpha=alpha, num_layers = num_layers,
                 num_neurons=num_neurons)
    
grid3 = GridSearchCV(subtask3_buildfn, param_grid=param_grid, cv=folds, refit=False)
grid3_result = grid3.fit(x_train, y_subtask3,
                        batch_size=batch_size, epochs=epochs)

# Unpack best params, retrain model
lr, decay, alpha, num_layers, num_neurons = [int(x) for x in grid3_result.best_params_.values()]
subtask3_best_model = subtask3_reg(lr, decay, alpha, num_layers, num_neurons)

subtask3_history = subtask3_best_model.fit(x_train, y_subtask3,
                                            batch_size=batch_size, epochs=10*epochs)

# Prediction #

In [ ]:
# Load the test features, make predictions, generate csv
# 1. Import data (load imputed if already done)
test_data = pd.read_csv('imputed_test_features.csv')
data_headers = pd.read_csv('train_labels.csv').columns
test_data = test_data.ix[:,1:] # Remove index column
#test_data = custom_impute(test_data)
pids_test, x_test = feature_transform(test_data)

y1 = subtask1_best_model.predict(x_test)
y2 = subtask2_best_model.predict(x_test)
y3 = subtask3_best_model.predict(x_test)
y_test_pred = np.concatenate([np.expand_dims(pids_test,axis=1), y1, y2, y3], axis=1)

df_test_pred = pd.DataFrame(y_test_pred, columns=data_headers)
df_test_pred.to_csv('prediction.csv', index=False, float_format='%.4f')